In [4]:
import cv2
import numpy as np

# --- Load ảnh và displacement map ---
wrapped_img = cv2.imread("/workspace/test_gen/images/000006.jpg")
displacement_map = np.load("/workspace/test_gen/labels/000006.npy")  # shape (H, W, 2)

H, W = displacement_map.shape[:2]

# --- 1. Scale displacement map về [-1,1] ---
map_x = displacement_map[..., 0]
map_y = displacement_map[..., 1]

map_x_norm = 2 * map_x / (W - 1) - 1   # map từ [0,W-1] -> [-1,1]
map_y_norm = 2 * map_y / (H - 1) - 1   # map từ [0,H-1] -> [-1,1]

disp_norm = np.stack([map_x_norm, map_y_norm], axis=-1)
print("Normalized displacement map range:", disp_norm.min(), disp_norm.max())

# --- 2. Giả lập predict ---
pred_norm = disp_norm.copy()  # dùng luôn để test

# --- 3. Rescale predicted map về tọa độ gốc pixel ---
map_x_pred = ((pred_norm[..., 0] + 1) / 2) * (W - 1)
map_y_pred = ((pred_norm[..., 1] + 1) / 2) * (H - 1)

disp_pred = np.stack([map_x_pred, map_y_pred], axis=-1).astype(np.float32)

# --- 4. Unwrap ảnh ---
def unwrap_image(wrapped_img, displacement_map):
    map_x, map_y = displacement_map[..., 0], displacement_map[..., 1]
    unwrapped_img = cv2.remap(
        wrapped_img, map_x, map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(255, 255, 255)
    )
    return unwrapped_img

unwrapped_img = unwrap_image(wrapped_img, disp_pred)
cv2.imwrite("unwrapped_test_neg1_1.png", unwrapped_img)
print("Unwrap xong, lưu ảnh ra 'unwrapped_test_neg1_1.png'")


Normalized displacement map range: -1.0 1.0003734
Unwrap xong, lưu ảnh ra 'unwrapped_test_neg1_1.png'


In [5]:
import cv2
import numpy as np

# --- Load ảnh và displacement map ---
wrapped_img = cv2.imread("/workspace/test_gen/images/000006.jpg").astype(np.float32)
displacement_map = np.load("/workspace/test_gen/labels/000006.npy")  # shape (H, W, 2)

H, W = displacement_map.shape[:2]

# --- 0. Scale ảnh input về [-1,1] ---
img_norm = (wrapped_img / 127.5) - 1
print("Normalized image range:", img_norm.min(), img_norm.max())

# --- 1. Scale displacement map về [-1,1] ---
map_x = displacement_map[..., 0]
map_y = displacement_map[..., 1]

map_x_norm = 2 * map_x / (W - 1) - 1
map_y_norm = 2 * map_y / (H - 1) - 1

disp_norm = np.stack([map_x_norm, map_y_norm], axis=-1)
print("Normalized displacement map range:", disp_norm.min(), disp_norm.max())

# --- 2. Giả lập predict ---
pred_norm = disp_norm.copy()  # dùng luôn để test

# --- 3. Rescale predicted map về tọa độ gốc pixel ---
map_x_pred = ((pred_norm[..., 0] + 1) / 2) * (W - 1)
map_y_pred = ((pred_norm[..., 1] + 1) / 2) * (H - 1)

disp_pred = np.stack([map_x_pred, map_y_pred], axis=-1).astype(np.float32)

# --- 4. Rescale ảnh input về gốc (nếu cần) ---
img_rescaled = ((img_norm + 1) * 127.5).astype(np.uint8)

# --- 5. Unwrap ảnh ---
def unwrap_image(wrapped_img, displacement_map):
    map_x, map_y = displacement_map[..., 0], displacement_map[..., 1]
    unwrapped_img = cv2.remap(
        wrapped_img, map_x, map_y,
        interpolation=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT,
        borderValue=(255, 255, 255)
    )
    return unwrapped_img

unwrapped_img = unwrap_image(img_rescaled, disp_pred)
cv2.imwrite("unwrapped_test_neg1_1_scaled.png", unwrapped_img)
print("Unwrap xong, lưu ảnh ra 'unwrapped_test_neg1_1_scaled.png'")


Normalized image range: -1.0 1.0
Normalized displacement map range: -1.0 1.0003734
Unwrap xong, lưu ảnh ra 'unwrapped_test_neg1_1_scaled.png'
